In [1]:
import numpy as np
import pandas as pd
import pyhere

In [2]:
dir_raw_data = pyhere.here().resolve().joinpath("data", "raw")


In [3]:
csv_power_plants = pd.read_csv(dir_raw_data/"global_power_plant_database.csv")

/tmp/ipykernel_1438/1573865454.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_power_plants = pd.read_csv(dir_raw_data/"global_power_plant_database.csv")


In [9]:
sample_lat_lon = csv_power_plants[["latitude", "longitude"]].head(2)


In [10]:
list_of_tuples = list(sample_lat_lon.to_records(index=False))
list_of_tuples


[(32.322, 65.119), (31.67, 65.795)]

# MULTIPOINT

In [89]:
'''
*Version: 2.0 Published: 2021/03/09* Source: [NASA POWER](https://power.larc.nasa.gov/)
POWER API Multi-Point Download
This is an overview of the process to request data from multiple data points from the POWER API.
'''

# TQV                   MERRA-2 Total Column Precipitable Water (kg m-2) 
# WS10M                 MERRA-2 Wind Speed at 10 Meters (m/s) 
# CLRSKY_SFC_SW_DNI     CERES SYN1deg Clear Sky Surface Shortwave Downward Direct Normal Irradiance (kW-hr/m^2/day) 


import os, sys, time, json, urllib3, requests, multiprocessing, csv
from io import StringIO

urllib3.disable_warnings()
df_response = pd.DataFrame()
def download_function(collection):
    ''' ''' 

    request, filepath, coordinates = collection
    response = requests.get(url=request, verify=False, timeout=30.00).json()
    # print(json.dumps(response["properties"], indent=2))
    # print(response["properties"])
    for obj in response["properties"].items():
        # df_response = pd.DataFrame.from_dict(obj["parameter"])
        print (obj["parameter"])
    # df_response = pd.DataFrame.from_dict(response["properties"])
    # print(df_response)
    # with open(filepath, 'w') as file_object:
        # json.dump(response, file_object)

def download_function_csv(collection):
    ''' '''

    request, filepath, coordinates = collection
    response = requests.get(url=request, verify=False, timeout=30.00).content.decode('utf-8')
    # print(pd.read_csv(StringIO(response)))
    df_response_aux = pd.read_csv(StringIO(response))
    df_response_aux["latitude"] = coordinates[0]
    df_response_aux["longitude"] = coordinates[1]

    if(df_response.empty):
        df_response = df_response_aux
    else:
        pd.concat([df_response,df_response_aux])
    
    print(df_response)
    # return df_response
    # with open(filepath, 'w') as file_object:
    #     file_object.write(response)
    #     file_object.close()

class Process():

    def __init__(self):

        self.processes = 5 # Please do not go more than five concurrent requests.

        self.request_template = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters=CLRSKY_SFC_SW_DNI,WS10M,TQV&community=RE&longitude={longitude}&latitude={latitude}&start=1990&end=2020&format=CSV&header=false"
        self.filename_template = "File_Lat_{latitude}_Lon_{longitude}.csv"

        self.messages = []
        self.times = {}

    def execute(self):

        Start_Time = time.time()

        locations = list_of_tuples

        requests = []
        for latitude, longitude in locations:
            request = self.request_template.format(latitude=latitude, longitude=longitude)
            filename = self.filename_template.format(latitude=latitude, longitude=longitude)
            coordinates = (latitude, longitude)
            requests.append((request, filename, coordinates))

        requests_total = len(requests)
        pool = multiprocessing.Pool(self.processes)
        x = pool.imap_unordered(download_function_csv, requests)
        for i, df in enumerate(x, 1):
            sys.stderr.write('\rExporting {0:%}'.format(i/requests_total))

        self.times["Total Script"] = round((time.time() - Start_Time), 2)

        print ("\n")
        print(df_response)
        print ("Total Script Time:", self.times["Total Script"])

if __name__ == '__main__':
    Process().execute()


32.322
65.119
Empty DataFrame
Columns: []
Index: []
31.67
65.795
Empty DataFrame
Columns: []
Index: []


Exporting 100.000000%



Empty DataFrame
Columns: []
Index: []
Total Script Time: 3.21


str